<a href="https://colab.research.google.com/github/newb-dev-1008/Blind-Object-Detection/blob/main/Live_Object_Detection_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>